[Pytorch Korea](https://tutorials.pytorch.kr/beginner/basics/quickstart_tutorial.html)

코드는 파이토치 코리아를 참고하여 만들었습니다.

더 알고 싶은 내용이 있다면 파이토치 코리아 링크를 따라가 보시기 바랍니다.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# TORCH의 DATASET

여기서는 토치 라이브러리에 저장된 데이터 셋(FashionMNIST)를 이용합니다.

토치는 데이터 셋에 대한 전처리를 도와주는 라이브러리를 제공하고 있습니다.

더 자세한 사항은 [DATASET](https://tutorials.pytorch.kr/beginner/basics/data_tutorial.html)을 참고해 보시기 바랍니다.



## ToTensor

<img src="https://drive.google.com/uc?export=view&id=16EL0AT_vlU5corNhN5HnPZ7lZuDCHLgw" height=300, width = 500>

이미지를 다루는 유명한 라이브러리들은 대부분 이미지 구조를 "H x W x C"로 이용합니다.
하지만 Torch에서는 "C x H x W"의 배열 구조를 이용하기 때문에 이미지 배열을 바꿔줘야 합니다.



1. 배열구조
"H x W x C"  →  "C x H x W"



2. 이미지 픽셀 밝기 정도(scale)

  [0\~255] >  [0~1]

In [2]:
 #공개 데이터셋에서 학습 데이터를 내려받습니다.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 공개 데이터셋에서 테스트 데이터를 내려받습니다.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:01<00:00, 14799103.17it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 272827.39it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5033397.00it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 20236435.79it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



## DATALOADER

우리는 대부분 SGD기반의 옵티마이저를 사용하기 때문에 data를 미니 배치로 나눠 줘야합니다.

이 일을 DataLoader가 배치로 나눈 데이터를 준비해 줍니다.


<img src="https://drive.google.com/uc?export=view&id=1VmCLogiQ_iG5TBlHz4_rU9f3DNVniCuz" height=300, width = 500>

---

<img src="https://drive.google.com/uc?export=view&id=1LfqmZTdGKFZuV9Sdy7osqYRCrk2mS7xZ" height=300, width = 500>



In [3]:
batch_size = 64

# 데이터로더를 생성합니다.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## 모델 만들기

이제 모델을 만들어 줄겁니다.

우리가 늘 배우고 있는 아키텍쳐를 구현하여 만들 수 있습니다. class를 정의하여 만드는 것이 기본이고 모델 class는 반드시 nn.Module을 상속하고 forward함수를 가지고 있어야합니다.

device라는 변수를 이용하여 gpu를 사용할지 cpu를 사용할지 결정해줍니다.

이후  to(device)를 이용하여 모델과 데이터를 gpu로 보내는 과정이 gpu를 사용한다면 꼭 필요합니다.

<img src="https://drive.google.com/uc?export=view&id=1j8Q7Cvum5PLt8bpcpbO_O8FGuyAB-blb" height=200, width = 200>


In [4]:
# 학습에 사용할 CPU나 GPU장치를 얻습니다.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

# 모델을 정의합니다.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## 모델 파라미터 최적화하기

우리의 목적에 맞는 loss function과 optimizer를 결정해줍니다.

우리는 현재 classification을 진행중이므로 cross entroopy를 이용해 보겠습니다.

optimizer는 쉽게 SGD를 이용합니다.

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

## TRAIN

이제 Train함수를 만들어 보겠습니다.

먼저  dataloader를 for문에 넣어서 미니 배치 하나를 받아옵니다.
미니 배치는 to(device)를 이용해 gpu로 넣어줍니다.

이제 모델을 이용해 입력 데이터의 prediction값을 얻고 loss function으로 실제 데이터와 prediction의 차이를 구합니다.

이 차이를 이용하여 우리는 back propagation을 시작합니다. 역전파를 시작하기 전에 optimizer.zero_grad()를 이용하여 변수들을 초기화 합니다. 이렇게 하지 않으면 이전 iteration의 변수(gradient)가 누적되어 제대로된 결과를 가져오기 어렵습니다.

loss.backward, optimizer.step은 이제 실제 역전파를 진행하는 함수라고 보면 될거 같네요.

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

## TEST

이제 모델을 평가해봅시다.
여기서는 model을 evaluation모드로 변경하고,
평가는 gradient를 계산할 필요가 없으므로

 with torch.no_grad()를 이용해 gradient계산 과정을 없애줍니다.

 다시 데이터를 gpu로 보내주고, 모델의 예측값을 꺼냅니다.

 이후는 모델의 평균 로스 그리고 정확도를 구해주는 식입니다.

In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

이제 에폭을 돌려볼까요!

In [8]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.308099  [   64/60000]
loss: 2.295934  [ 6464/60000]
loss: 2.278307  [12864/60000]
loss: 2.267631  [19264/60000]
loss: 2.251933  [25664/60000]
loss: 2.225580  [32064/60000]
loss: 2.234968  [38464/60000]
loss: 2.198143  [44864/60000]
loss: 2.191296  [51264/60000]
loss: 2.162457  [57664/60000]
Test Error: 
 Accuracy: 45.1%, Avg loss: 2.158550 

Epoch 2
-------------------------------
loss: 2.168748  [   64/60000]
loss: 2.160011  [ 6464/60000]
loss: 2.102834  [12864/60000]
loss: 2.110499  [19264/60000]
loss: 2.067503  [25664/60000]
loss: 2.008153  [32064/60000]
loss: 2.039857  [38464/60000]
loss: 1.954283  [44864/60000]
loss: 1.951712  [51264/60000]
loss: 1.881391  [57664/60000]
Test Error: 
 Accuracy: 56.0%, Avg loss: 1.880738 

Epoch 3
-------------------------------
loss: 1.912465  [   64/60000]
loss: 1.883477  [ 6464/60000]
loss: 1.764920  [12864/60000]
loss: 1.798863  [19264/60000]
loss: 1.699602  [25664/60000]
loss: 1.647169  [32064/600

## 모델 저장하기

우리가 생성한 모델은 저장 할 수 있습니다.
path를 정해주면 거기에 저장됩니다.

In [9]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## 모델 불러오기

저장된 모델을 다시 불러와 보죠!

모델을 불러오는 과정에는 모델 구조를 다시 만들고 상태 사전을 모델에 불러오는 과정이 포함됩니다.

In [10]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [11]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
